# **D&D Spellbook Network Analysis**

Installing pyvis

In [61]:
!pip install pyvis

Importing Pandas and network

In [62]:
import pandas as pd
import networkx as nx

Loading dataset into pandas

In [63]:
df = pd.read_csv('dnd-spells.csv')
df.head()

,name,classes,level,school,cast_time,range,duration,verbal,somatic,material,material_cost,description
0,Acid Splash,"Artificer, Sorcerer, Wizard",0,Conjuration,1 Action,60 Feet,Instantaneous,1,1,0,NaN,You hurl a bubble of acid. Choose one creature...
1,Blade Ward,"Bard, Sorcerer, Warlock, Wizard",0,Abjuration,1 Action,Self,1 round,1,1,0,NaN,You extend your hand and trace a sigil of ward...
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard",0,Evocation,1 Action,Self (5-foot radius),1 round,0,1,1,a melee weapon worth at least 1 sp,You brandish the weapon used in the spell’s ca...
3,Chill Touch,"Sorcerer, Warlock, Wizard",0,Necromancy,1 Action,120 Feet,1 round,1,1,0,NaN,"You create a ghostly, skeletal hand in the spa..."
4,Control Flames,"Druid, Sorcerer, Wizard",0,Transmutation,1 Action,60 Feet,Instantaneous or 1 hour,0,1,0,NaN,You choose nonmagical flame that you can see w...


Seperating classes and ignoring cloumns we dont need

In [64]:
df['classes_list'] = df['classes'].str.split(', ')
columns_to_keep = ['name', 'classes', 'classes_list', 'school']
df = df[columns_to_keep]
df.head()

,name,classes,classes_list,school
0,Acid Splash,"Artificer, Sorcerer, Wizard","[Artificer, Sorcerer, Wizard]",Conjuration
1,Blade Ward,"Bard, Sorcerer, Warlock, Wizard","[Bard, Sorcerer, Warlock, Wizard]",Abjuration
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard","[Artificer, Sorcerer, Warlock, Wizard]",Evocation
3,Chill Touch,"Sorcerer, Warlock, Wizard","[Sorcerer, Warlock, Wizard]",Necromancy
4,Control Flames,"Druid, Sorcerer, Wizard","[Druid, Sorcerer, Wizard]",Transmutation


Graph Creation
1. add spells as nodes.
2. using school as a basis to apply different color on spells.
3. Add edges for spells with atleast 4 common class.

In [66]:

 G = nx.Graph()

for index, row in df.iterrows():
    G.add_node(row['name'], school=row['school'])

spells = list(df['name'])
for i in range(len(spells)):
    for j in range(i + 1, len(spells)):
        spell1_name = spells[i]
        spell2_name = spells[j]
        spell1_classes = df.loc[df['name'] == spell1_name, 'classes_list'].iloc[0]
        spell2_classes = df.loc[df['name'] == spell2_name, 'classes_list'].iloc[0]

        common_classes = set(spell1_classes) & set(spell2_classes)

        if len(common_classes) >= 4:
            G.add_edge(spell1_name, spell2_name)

print(f"Refined Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

Refined Graph created with 554 nodes and 1644 edges.


Create a dictionary to map each school of magic to a color

In [67]:
# Find all the unique values in the 'school' column
unique_schools = df['school'].unique()

print("All unique schools of magic found in the dataset:")
print(unique_schools)

color_palette = [
    '#ff69b4', '#00bfff', '#ff4500', '#dc143c',
    '#9932cc', '#006400', '#ffd700', '#ffffff'
]

# This pairs each school with a color from our palette
color_map = {school: color for school, color in zip(unique_schools, color_palette)}

print("color map:")
for school, color in color_map.items():
    print(f"{school}: {color}")

for node in G.nodes():
    school = G.nodes[node]['school']
    G.nodes[node]['color'] = color_map.get(school, '#cccccc')

All unique schools of magic found in the dataset:
['Conjuration' 'Abjuration' 'Evocation' 'Necromancy' 'Transmutation'
 'Enchantment' 'Divination' 'Illusion']
color map:
Conjuration: #ff69b4
Abjuration: #00bfff
Evocation: #ff4500
Necromancy: #dc143c
Transmutation: #9932cc
Enchantment: #006400
Divination: #ffd700
Illusion: #ffffff


In [68]:
from pyvis.network import Network

net_virtual = Network(
    notebook=True,
    height="800px",
    width="100%",
    bgcolor="#222222",
    font_color="white",
    cdn_resources='in_line'
)


net_virtual.from_nx(G)

net_virtual.solver = "forceAtlas2Based"
net_virtual.force_atlas_2based(gravity=-80, spring_length=150)

# Save the final colored graph
net_virtual.save_graph("spell_network.html")

print("Final colored graph 'spell_network.html' has been saved. Please download and open it.")

Final colored graph 'spell_network.html' has been saved. Please download and open it.
